# Linear Econometric Models

So far in our Scientific Python adventures we have covered:

* NumPy
* SciPy - optimization
* Pandas
* Matplotlib (in your own time)

For our final adventure into Python's Scientific stack we are going to look into estimating simple linear econometric models. This is just the tip of the iceberg as far as statistical modelling packages and capabilities in Python - but it is enough to get you hopefully confident enough to go looking for further modelling libraries on your own.

This notebook is structured a little bit like how a simple project might look. We do all the tasks needed from downloading the data, some data visualization and the econometrics together. This should show you how far we have come on our Python Adventure. 

## Getting and Loading some data

The plan is to use the data from AJR01's paper on linking institutions to ecconomic development as an example on how to run OLS and IV style regression models. That means we have to load the data.

Let's use Python to download the file, and import it into our session as a Pandas DataFrame. Unfortunately Acemoglu's own data repository is weirdly formatted - so the work we are gonna do here is a little frustrating, but a good exercise in using what we may have learned last week with Julian.

#### Downloading the data

In [ ]:
import shutil
import requests
import zipfile

In [ ]:
url = 'https://economics.mit.edu/files/5197'
data_zip = '../data/table1.zip'

In [ ]:
# connect to the page
response = requests.get(url, stream=True, verify=False)

# save the zip to our preferred location
with open(data_zip, 'wb') as out_file:
    shutil.copyfileobj(response.raw, out_file)

# clean up    
del response

Let's verify the data is located where we expected it to go:

In [ ]:
! ls ../data

#### Importing into Pandas

Unfortunately only `read_csv` comes with a built in option to read compressed data, so we have to do a bit of work here:

In [ ]:
import pandas as pd
import os
import glob
data_dir = '../data/'

First we need to extract the data from the zip file we just downloaded

In [ ]:
zip_ref = zipfile.ZipFile(data_zip, 'r')
zip_ref.extractall(data_dir)

In [ ]:
! ls ../data

The data set we want has the file ending `table1.dta`, so we need to get that value

In [ ]:
data_file = glob.glob(data_dir+'*table1.dta')[0]

Read in the data with pandas `read_stata` function

In [ ]:
data = pd.read_stata(data_file)

In [ ]:
data.head()

## Data Visualization

AJR's main hypothesis is to investigate the relationship between institutional differences and economic outcomes.

To do this the paper operationalizes the variables of interest as follows:

* economic outcomes are proxied by log GDP per capita in 1995, adjusted for exchange rates
* institutional differences are proxied by an index of protection against expropriation on average over 1985-95, constructed by the Political Risk Services Group

Before we jump on to running regressions, let's check out whether there is any relationship between these variables graphically:

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('seaborn-notebook')

In [ ]:
data.plot(x='avexpr', y='logpgp95', kind='scatter')
plt.show()

So there definitely appears to be a positive relationship. Next we could try and do a little more with this graph and plot a line through the data:

In [ ]:
import numpy as np
from statsmodels.nonparametric.smoothers_lowess import lowess


# Dropping NA's is required to use numpy's polyfit
df1_subset = data.dropna(subset=['logpgp95', 'avexpr'])

# Use only 'base sample' for plotting purposes
df1_subset = df1_subset[df1_subset['baseco'] == 1]

X = df1_subset['avexpr']
y = df1_subset['logpgp95']
labels = df1_subset['shortnam']

# Replace markers with country labels
plt.scatter(X, y, marker='')

for i, label in enumerate(labels):
    plt.annotate(label, (X.iloc[i], y.iloc[i]))

# Fit a linear trend line
plt.plot(np.unique(X),
         np.poly1d(np.polyfit(X, y, 2))(np.unique(X)),
         color='black')

plt.xlim([3.3,10.5])
plt.ylim([4,10.5])
plt.xlabel('Average Expropriation Risk 1985-95')
plt.ylabel('Log GDP per capita, PPP, 1995')
plt.title('Relationship between expropriation risk and income')
plt.show()

## Univariate Regression

OK, so now we have visualized data let's run some OLS regressions. We will start with the simplest regression we can think of:

$$
logpgp95_i = \beta_0 + \beta_1 avexpr + u_i
$$


First we need to load the `statsmodels` package:

In [ ]:
import statsmodels as sm
sm.__version__

There's two ways to write out the specification for a regression, the easiest is by writing a formulaic description of the regression we want to estimate:

In [ ]:
import statsmodels.formula.api as smf

In [ ]:
simple_reg = smf.ols('logpgp95 ~ avexpr', data=data)

To estimate the regression specification, we use the `fit` method:

In [ ]:
results = simple_reg.fit()

The `results` object now contains a bunch of information. We can see what's inside (the names of) as follows:

In [ ]:
#dir(results)

Most of the 'standard output' we have come to expect from software like STATA are contained in the `.summary()`:

In [ ]:
print(results.summary())

An observation was mistakenly dropped from the results in the original paper (see the note located in maketable2.do from Acemoglu’s webpage), so our coefficients differ slightly from the ones in the original paper.

We can get the predicted values from the regression using the `.predict()` attribute. Let's use these in a plot that has predicted versus actual values:

In [ ]:
# Drop missing observations from whole sample

df_plot = data.dropna(subset=['logpgp95', 'avexpr'])

# Plot predicted values

plt.scatter(df_plot['avexpr'], results.predict(), alpha=0.5, color='red', label='predicted')

# Plot observed values
plt.scatter(df_plot['avexpr'], df_plot['logpgp95'], alpha=0.5, label='observed')

plt.legend()
plt.title('Predicted vs actual values for simple OLS regression')
plt.xlabel('avexpr')
plt.ylabel('logpgp95')
plt.show()

We mentioned that there is two ways to call a linear regression, and so far we showed how to use the formula interface which is probably the recommended way to do this in most applications.

If we end up in a situation where we have to run a regression many times the formula interface actually slows down the code because python has to deconstruct it to get the exact information it needs. Examples where this may occur (and I have come across) are if a regression is part of a nested loop over structural parameters (BLP estimation for IO people), or if we are coding up our own indirect inference procedure.

Here's another way to get the same result that is usually faster, although we have to import some additional functionality:

In [ ]:
from statsmodels.tools.tools import add_constant

data = add_constant(data, has_constant='add')

In [ ]:
data.columns

In [ ]:
from statsmodels.regression.linear_model import OLS

In [ ]:
simple_reg2 = OLS(endog=data['logpgp95'], 
                     exog=data[['const', 'avexpr']], 
                     missing='drop')
type(simple_reg2)

In [ ]:
print(simple_reg2.fit().summary())

Notice that this gives us exactly the same results.

## Multivariate Regression

Univariate regression is rarely enough in social science. At a bare minimum we are worried about other factors affecting GDP that are not included in our model that are also correlated with institutional quality. As reasonably good econometricians we know that this causes bias / inconsistency in our estimates for $\hat{\beta}$.

So let's add more regressors! AJR argue that climate influences economic outcomes, and it is probably correlated to institutional quality (Discuss / think about why). They operationalize this by including latitude as a proxy. 

Notice that our data doesn't have that variable due to the 'interesting' way Acemoglu structures his data and do files on the web. We are going to have to pull another data set. We've created a simple function to make our lives easier:

In [ ]:
# import from another directory

import sys
sys.path.append('../')

from lib import ajr

weblink      = 'https://economics.mit.edu/files/5135'
data_dir     = '../data/'
zip_name     = 'table2.zip'
table_number = 'table2'

data = ajr.download_data(weblink, data_dir, zip_name, table_number)

Now, using the formula interface we can readily add this variable:

In [ ]:
reg2 = smf.ols('logpgp95 ~ avexpr + lat_abst', data=data)
results2 = reg2.fit() 

In [ ]:
print(results2.summary())

This is nice - but its a little hard to compare coefficients and the like across regressions that are far apart in the notebook. What we really want is a regression table that summarizes all of our results. `statsmodels` allows us to do this too:

In [ ]:
from statsmodels.iolib.summary2 import summary_col

info_dict={'R-squared' : lambda x: "{:.2f}".format(x.rsquared),
           'No. observations' : lambda x: "{0:d}".format(int(x.nobs))}

results_table = summary_col(results=[results,results2],
                            float_format='%0.2f',
                            stars = True,
                            model_names=['Model 1',
                                         'Model 2'],
                            info_dict=info_dict,
                            regressor_order=['avexpr',
                                             'lat_abst',
                                             'const'])

results_table.add_title('Table - OLS Regressions')

print(results_table)

If we decide we want this table as latex output we can do that too:

In [ ]:
with open('../../out/tables/ols_summary.tex', 'w') as file_handle:
    file_handle.write(results_table.as_latex())

### Challenge

It's your turn!

1. Add the following variables to the regression using the formula interface:
    * 'asia', 'africa', 'other'
2. Add the same variables as (1); but use the alternative interface that requires you to specify 'endog' and 'exog' variables
3. Export the results to a latex file 'out/tables/ols-regressions'. Give the columns meaningful names, and make sure you don't have stars in your output to satisfy the new AER policy.

## Robustifying Standard Errors

So far our regressions have imposed heteroskedastic standard errors.

A simple test for Heteroskedasticity is the Breusch Pagan test:

In [ ]:
reg3 = smf.ols('logpgp95 ~ avexpr + lat_abst + asia + africa + other', data=data)
results3 = reg3.fit() 

from statsmodels.compat import lzip
import statsmodels.stats.api as sms


In [ ]:
name = ['Lagrange multiplier statistic', 'p-value', 
        'f-value', 'f p-value']
test = sms.het_breushpagan(results3.resid, results3.model.exog)
lzip(name, test)

There is some evidence for heteroskedasticity. Let's robustify standard errors. Statsmodels has 'HC0' 'HC1', 'HC2' and 'HC3' standard errors. The STATA default is HC1, so for better or worse let's geta a summary with those:

In [ ]:
robust_ols = reg3.fit(cov_type='HC1')
print(robust_ols.summary())

There are also options for clustered standard errors. A nice summary of what is available albiet on a different data set is available [here](http://www.vincentgregoire.com/standard-errors-in-python/)

## Instrumental Variables Estimation

AJR discuss that the OLS regression results may suffer from endogeneity due to:


* richer countries may be able to afford or prefer better institutions
* variables that affect income may also be correlated with institutional differences
* the construction of the index may be biased; analysts may be biased towards seeing countries with higher income having better institutions

They propose a 2SLS solution using settler mortality to instrument for institutional differnences: They hypothesize that higher mortality rates of colonizers led to the establishment of institutions that were more extractive in nature (less protection against expropriation), and these institutions still persist today.

A scatter plot provides some evidence towards the necessary condition of a correlation between the instrument and the endogenous variable:

In [ ]:
weblink      = 'https://economics.mit.edu/files/5136'
data_dir     = '../data/'
zip_name     = 'table4.zip'
table_number = 'table4'

data = ajr.download_data(weblink, data_dir, zip_name, table_number)
data.head()

In [ ]:
# Dropping NA's is required to use numpy's polyfit
df1 = data.dropna(subset=['logem4', 'avexpr'])

X = df1['logem4']
y = df1['avexpr']
labels = df1['shortnam']

# Replace markers with country labels
plt.scatter(X, y, marker='')

for i, label in enumerate(labels):
    plt.annotate(label, (X.iloc[i], y.iloc[i]))

# Fit a linear trend line
plt.plot(np.unique(X),
         np.poly1d(np.polyfit(X, y, 1))(np.unique(X)),
         color='blue')

plt.xlim([1.8,8.4])
plt.ylim([3.3,10.4])
plt.xlabel('Log of Settler Mortality')
plt.ylabel('Average Expropriation Risk 1985-95')
plt.title('First-stage relationship between settler mortality and expropriation risk')
plt.show()

Let's estimate the first stage relationship for the univariate regression model:

In [ ]:
data = data.query('baseco ==1')

first_state_reg = smf.ols('avexpr ~ logem4', 
                          data=data)

In [ ]:
print(first_state_reg.fit(cov_type='HC1').summary())

So the first stage seems OK with a t-state > 3 (approx F > 10). Let's do an IV2SLS estimation using the `linearmodels` package:

In [ ]:
from linearmodels.iv import IV2SLS

In [ ]:
iv_regression = IV2SLS.from_formula('logpgp95 ~ 1 + [avexpr ~ logem4]', 
                                      data=data).fit()

In [ ]:
iv_regression.summary

### Final Challenge

1. Estimate the IV model for the regression that includes 'asia', 'africa' and 'other' as exogenous explanatory variables
2. Produce a regression table that compares the 4 coefficient estimates on 'avexpr', all using robust standard errors.
3. Use a robust Hausman test to conduct a formal test for endogeneity. Was the IV strategy appropriate based on this test result?